#### Felix Adam - Computing Lab Assignment 2 -  Webscraping Beer Prices

The goal of this notebook is to scrape current beer prices from a big german food delivery site. 
Secondary goal: Find beers that are on sale!

1. Make use of beautifoul soup 
2. Implement the whole process with ques!



In [322]:
# Importing Libraries
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
import queue
from concurrent.futures import ThreadPoolExecutor


# Selenium Options, we want to use a "headless" driver, so no windows pop up
browser_options = Options()
browser_options.add_argument("--headless")

# Setting Webdriver to Firefox
driver = webdriver.Firefox(options=browser_options)

In [317]:
## Get Beer
def get_beer(beer_url):
    # Getting Request
    driver.get(beer_url)
    
    # Parsing with beautiful soup
    beer_parsed = BeautifulSoup(driver.page_source, 'lxml')

    # Title
    beer_title = beer_parsed.find('h1', class_= 'pd-QuickInfo__heading' ).text

    # Price, integer part
    integer_price = beer_parsed.find('span', class_='pd-price__predecimal').text

    # Price, decimal part, removing leading whitespace as well
    decimal_price = beer_parsed.find('span', class_='pd-price__decimal').text.lstrip()

    # Full Price, concat and convert to float
    beer_price = float(integer_price+'.'+decimal_price)

    #Get the info string
    info_string = beer_parsed.find('div', class_='rs-qa-price-base pd-Grammage pd-Grammage--Detail').text

    # Extract info inside of the brackets
    price_per_liter_string = info_string[info_string.find("(")+1:info_string.find(")")]

    # Extract string left to the equal sign (e.g. 1 l = 2,17 € or 1 l = 2 €)
    temp_price_liter = price_per_liter_string.split('=')[1]
    
    # Error handling for missing commas
    if "," not in temp_price_liter:
        price_per_liter = int(re.search(r'\d+', temp_price_liter).group())

    else:
        # Get the decimal 
        temp_price_liter_int  = temp_price_liter.split(',')[0]
    
        # Get the integer
        temp_price_liter_dec = temp_price_liter.split(',')[1].split(' ')[0]

        price_per_liter = float(temp_price_liter_int+'.'+temp_price_liter_dec)
    
        # Alcohol Content in percent adapt later!
        #alcohol_content = beer_parsed.find_all('div', class_='pd-Attribute')[2].text.split(' ')[1]

        # Collect all the beer info in a list
        beer = [beer_title, beer_price, price_per_liter] 
    
    #Return the beer
    return beer
    
def get_beer_list(list_url):
    
    #beers_to_get = queue.Queue()
    beer_list = []
    
    # Trying to get the page, except connfection refused (to many requests!)
    beer_list_page = driver.get(list_url)
    
    # Parsing with beautiful soup
    beer_list_page = BeautifulSoup(driver.page_source, 'lxml')
    
    # Get all beers on the page
    beers = beer_page.find_all('div', class_= 'search-service-ProductTileContent')
    
    for beer in beers:
        beer_url = 'https://shop.rewe.de'+beer.find('a')['href']
        
        beer_list.append(beer_url)
        #beers_to_get.put(beer_url)
    
    return beer_list

 

In [320]:
get_beer('https://shop.rewe.de/p/eichbaum-ureich-premium-pils-0-5l/1447984')

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=54925): Max retries exceeded with url: /session/7db5d4c9-bbe1-4e3a-b614-cabf24899536/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa673962f98>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [321]:
try:
    beer_list_page = driver.get('https://shop.rewe.de/c/getraenke/?search=bier')
except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=54925): Max retries exceeded with url: /session/7db5d4c9-bbe1-4e3a-b614-cabf24899536/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa674483cc0>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [302]:
# Start URL : https://shop.rewe.de/c/getraenke/?search=bier
##Get all links ##

beers_to_get = queue.Queue()

# Get page
#driver.get('https://shop.rewe.de/c/getraenke-bier-mischgetraenke/?search=Bier')
    
# Parsing with beautiful soup
beer_page = BeautifulSoup(driver.page_source, 'lxml')

beers = beer_page.find_all('div', class_= 'search-service-ProductTileContent')

for beer in beers:
    beer_url = 'https://shop.rewe.de'+beer.find('a')['href']
    beers_to_get.put(beer_url)


In [316]:
beer_page.find('div', class_='search-service-pagination--flex')



In [313]:
beer_page.find_all('a', class_,href = True)

[<a href="https://www.rewe.de/service/datenschutz/" title="Details ansehen">Details ansehen</a>,
 <a class="ths-cookie-notification__confirm-trigger ths-js-cookie-notification-confirm-trigger" href="#">hier</a>,
 <a class="ths-mobile-header__logo ths-qa-mobile-header-logo" href="/" title="Zur Startseite vom REWE Lieferservice"> REWE Lieferservice
     </a>,
 <a class="rs-shoppinglist-mobile-btn" href="/shoppinglist"><svg>
 <use xlink:href="#sl_icon_heart" xmlns:xlink="http://www.w3.org/1999/xlink"></use>
 </svg></a>,
 <a class=" basket-link-mobile " data-basket-link="" href="/checkout/start" id="rs-qa-basket-triggerMobil" rel="nofollow"><span class="basket-link_icon"><span class="basket-link_icon--amount" data-basket-link-amount="">0</span><img class="basket-link_icon--icon_empty" src="https://shop.rewe-static.de/rewe-basket/47c3b5b54/images/basket-link-mobile.svg"/><img class="basket-link_icon--icon_filled" src="https://shop.rewe-static.de/rewe-basket/47c3b5b54/images/basket-link-acti

In [ ]:
def worker(i):
    while True:
        item = q.get()
        if item == 'break':
            break
        results = do_work(item)
        if results:
            q.put(r) for r in results
        q.task_done()

q = queue.Queue()

q.put(starting_thing)


# stop workers
with ThreadPoolExecutor(50) as pool:
    futures = pool.map(worker, range(50))

    # block until all tasks are done
    q.join()
    
    # Tell all our workers to stop
    for i in range(50):
        q.put('break')

In [229]:
# Getting Beers multithreaded
q = queue.Queue()

q.put(beer_url)